### Monte carlo simulation to aggregate at the goal level


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# show all output from cells
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"  # last_expr

In [4]:
indicators = pd.read_csv("../output/indicators.csv", index_col=0)
indicators.set_index(["Country", "Year"], inplace=True)
indicators = indicators[[i for i in indicators if i.startswith("14")]]
indicators

14.1       14.3       14.5       14.7       14.b
Country        Year                                                       
Belgium        2012  47.872645  73.370139  87.232641  46.200186  79.053327
               2013  48.789672  73.474526  86.048292  52.202161  79.163392
               2014  37.048287  75.883383  85.798948  52.808338  79.138379
               2015  36.981616  74.454387  85.778589  52.449158  79.023306
               2016  51.278646  75.098105  85.055422  51.491526  78.778095
...                        ...        ...        ...        ...        ...
United Kingdom 2017  55.796762  87.862081  84.925435  69.610536        NaN
               2018  55.404844  88.511220  84.489188  71.031424        NaN
               2019  55.404844  89.605287  84.007510  70.897766        NaN
               2020  55.404844  92.172006  85.243933  69.011947        NaN
               2021  55.404844  92.172006  85.289779  69.378426        NaN

[220 rows x 5 columns]

In [8]:
# issue with extremely small e values
alpha = 1 / len(indicators.columns)
testArray = np.array([47.87264467, 72.99410253, 87.23264127, 46.2001862, 79.05332667])
e = 4.075778705425748e-05
((e - 1) / e)
(alpha * testArray ** ((e - 1) / e))
(e / (e - 1))
sum((alpha * testArray ** ((e - 1) / e))) ** (e / (e - 1))

-24534.188789047417

array([0., 0., 0., 0., 0.])

-4.075944831917252e-05

/tmp/ipykernel_46193/950930520.py:8: RuntimeWarning: divide by zero encountered in scalar power
  sum((alpha * testArray ** ((e - 1) / e))) ** (e / (e - 1))


inf

In [9]:
%%time
alpha = 1 / len(indicators.columns)


def compositeInd(alpha, indicators, sigma):
    # calculate the generalized mean
    composite = sum(alpha * indicators ** ((sigma - 1) / sigma)) ** (
        sigma / (sigma - 1)
    )
    return composite

# define seed to reproduce, random uniform elasticity, and number of simulations
np.random.seed(8)
simulations = 10_000
elasticity = np.random.uniform(0, 1, simulations)

scoresGoal = pd.DataFrame(columns=["country","year", "mean", "std"])
# loop through all countries and years, calculate the composite with different elasticity, return mean and std
years = [2012, 2016, 2021]
for year in years:
    for country in indicators.index.get_level_values(0).unique():
        scoreMC = np.array([])
        indicatorArray = indicators.loc[country, year].to_numpy()
        #ignore divide by zero error
        with np.errstate(divide='ignore'):
            # loop through all elasticity values, append to list of scores
            for e in elasticity:
                score = compositeInd(
                    alpha=1 / len(indicatorArray), indicators=indicatorArray, sigma=e
                )
                scoreMC = np.append(scoreMC, score)
                scoreMC = scoreMC[~np.isinf(scoreMC)] # remove inf values   
                
        # calculate mean and std, append to dataframe
        mean, std = np.mean(scoreMC), np.std(scoreMC)
        scoresGoal.loc[len(scoresGoal)] = [country, year, mean, std]
scoresGoal.set_index(["country", "year"], inplace=True)

CPU times: user 31.1 s, sys: 884 µs, total: 31.1 s
Wall time: 31.1 s


In [12]:
scoresGoal[scoresGoal.index.isin([2012], level=1)]

,,mean,std
country,year,,
Belgium,2012,60.242679,4.721797
Bulgaria,2012,NaN,NaN
Croatia,2012,NaN,NaN
Cyprus,2012,NaN,NaN
Denmark,2012,61.892955,6.562379
Estonia,2012,61.940891,3.853387
Finland,2012,62.712639,3.676606
France,2012,74.913291,2.985510
Germany,2012,74.920631,0.930454


In [11]:
# scores = pd.DataFrame(columns=["scores"])
# for year in years:
#     score = compositeInd(indicators=indicators[indicators.index.isin([year], level=1)].to_numpy())
#     score = pd.DataFrame(score, columns=["scores"])
#     scores = pd.concat([scores, score])
# scores
# scores.index = pd.MultiIndex.from_tuples(scores.index)
# scores